In [1]:
from cloudvolume import CloudVolume
import numpy as np

In [2]:
vol_im = CloudVolume("file:///cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/precomputed/ch1_iso")
vol_axons = CloudVolume("file:///cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/precomputed/axons")

In [ ]:
from cloudvolume.exceptions import SkeletonDecodeError
for i in range(66):
    try:
        verts = vol_axons.skeleton.get(i).vertices
    except SkeletonDecodeError:
        continue
    points = np.array(np.divide(verts, vol_axons.resolution))
    points = [list(c) for c in points]

    print(f"ID: {i} w/{len(points)} points")
    if len(points) <= 10000:
        with open("/cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/points/map_axons/target_space/" + str(i) + ".txt", "w") as f:
            for point in points:
                f.write(f"({point[0]},{point[1]},{point[2]})")
                f.write("\n")

In [ ]:
from cloudreg.scripts.transform_points import NGLink
from cloudreg.scripts.visualization import create_viz_link_from_json

viz_link_init = "https://viz.neurodata.io/?json_url=https://json.neurodata.io/v1?NGStateID=7m7rEoVJdsNt1Q"

viz = NGLink(viz_link_init.split("json_url=")[-1])
ngl_json = viz._json
ngl_json['layers'].append(
    {
        "type": "annotation",
        "annotations": points,
        "name" : "test_points"
    }
)
viz_link = create_viz_link_from_json(ngl_json)
print(viz_link)

python -m cloudreg.scripts.transform_points --target_viz_link https://viz.neurodata.io/?json_url=https://json.neurodata.io/v1?NGStateID=x4nz2KzMfzinTQ --atlas_viz_link https://ara.viz.neurodata.io/?json_url=https://json.neurodata.io/v1?NGStateID=HvyNDGaPsd1wyg --affine_path /cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/precomputed_ch1_otsu_iso_registration/downloop_1_A.mat --velocity_path /cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/precomputed_ch1_otsu_iso_registration/downloop_1_v.mat --soma_path /cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/path_coords.txt

### Write new skeleton layer

Copy and paste info files 

In [12]:
vol_axons_atlas = CloudVolume("file:///cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/precomputed/axons_atlas", compress=False)
vol_axons_atlas.chunk_size

Vec(128,128,128, dtype=int64)

### Read transformed points

In [22]:

from os import listdir
from os.path import isfile, join
import json

dir = '/cis/home/tathey/projects/mouselight/axon_mapping/low_res/2018-12-01/points/map_axons/atlas_space'
names = [f for f in listdir(dir) if isfile(join(dir, f))]
nums = [int(n.split('points')[-1][:-5]) for n in names]
files = [join(dir,f) for f in names]

for num,file in zip(nums, files):
    skel_targ = vol_axons.skeleton.get(num)
    skel_atlas = vol_axons_atlas.skeleton.get(num)
    print(f"Length of og skel: {len(skel_targ.vertices)} {type(skel_targ.vertices)} {skel_targ.vertices.dtype}")
    with open(file) as f:
        json_file = json.load(f)
        print(f"Length of transformed points: {len(json_file)}")
        coords = []
        for point in json_file:
            new_coord = np.multiply(point['point'], vol_axons_atlas.resolution)
            coords.append(new_coord)
        
        coords = np.array(coords, dtype='float32')
        print(type(coords))
        print(coords.dtype)
        skel_atlas.vertices = np.array(coords)
        #vol_axons_atlas.skeleton.upload(skel_atlas)
        vol_axons_atlas.skeleton.upload_raw(num, skel_atlas.vertices, skel_atlas.edges, skel_atlas.radii, skel_atlas.vertex_types)
        
    

Length of og skel: 2626 <class 'numpy.ndarray'> float32
Length of transformed points: 2626
<class 'numpy.ndarray'>
float32
Length of og skel: 2309 <class 'numpy.ndarray'> float32
Length of transformed points: 2309
<class 'numpy.ndarray'>
float32
Length of og skel: 5938 <class 'numpy.ndarray'> float32
Length of transformed points: 5938
<class 'numpy.ndarray'>
float32
Length of og skel: 2235 <class 'numpy.ndarray'> float32
Length of transformed points: 2235
<class 'numpy.ndarray'>
float32
Length of og skel: 536 <class 'numpy.ndarray'> float32
Length of transformed points: 536
<class 'numpy.ndarray'>
float32
Length of og skel: 488 <class 'numpy.ndarray'> float32
Length of transformed points: 488
<class 'numpy.ndarray'>
float32
Length of og skel: 688 <class 'numpy.ndarray'> float32
Length of transformed points: 688
<class 'numpy.ndarray'>
float32
Length of og skel: 1399 <class 'numpy.ndarray'> float32
Length of transformed points: 1399
<class 'numpy.ndarray'>
float32
Length of og skel: 517

In [23]:
skel = vol_axons_atlas.skeleton.get(1)
print(skel)
print(type(skel.vertices))
print(skel.vertices.dtype)
print(np.divide(skel.vertices, vol_axons_atlas.resolution))

Skeleton(segid=1, vertices=(shape=2626, float32), edges=(shape=2625, uint32), radius=(2626, float32), vertex_types=(2626, uint8), space='physical' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]])
<class 'numpy.ndarray'>
float32
[[738.6179   417.2736   391.523575]
 [738.6256   417.139225 391.283475]
 [741.68365  417.369675 389.83915 ]
 ...
 [743.9647   399.8301   412.92095 ]
 [743.1246   400.015425 413.931775]
 [748.3416   398.951125 404.07065 ]]


In [11]:
skel = vol_axons.skeleton.get(1)
print(skel)
print(type(skel.vertices))
print(skel.vertices.dtype)
print(np.divide(skel.vertices, vol_axons.resolution))

Skeleton(segid=1, vertices=(shape=2626, float32), edges=(shape=2625, uint32), radius=(2626, float32), vertex_types=(2626, uint8), space='physical' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]])
<class 'numpy.ndarray'>
float32
[[195.7661849  200.37846354  89.96969531]
 [195.64919271 200.32597656  89.96969531]
 [194.9371224  200.38518229  90.43039063]
 ...
 [205.75113281 192.69251302  90.74221875]
 [206.24449219 192.77044271  90.61816406]
 [201.46299479 192.37635417  91.38089063]]


In [ ]:
np.divide(vol_axons.skeleton.get(1).vertices, [10000,10000,10000])

In [ ]:
np.divide(vol_axons.skeleton.get(1).vertices, vol_axons.resolution)